<a href="https://www.kaggle.com/code/aisuko/computing-embeddings-streaming?scriptVersionId=162452512" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

In the notebook [Computing Embeddings with Multi-GPUs](https://www.kaggle.com/code/aisuko/computing-embeddings-with-multi-gpus), we did the computing of embeddings by using multiple GPUs. In this notebook, we want to add streaming data. Let's see the distribution computing+streaming data which is helpful in limit memory. And with streaming data, we don't need to wait for an extremely large dataset to download. See more detail about the streaming data on dataset at https://huggingface.co/docs/datasets/stream

In [ ]:
!pip install sentence-transformers==2.3.1
!pip install datasets==2.15.0
!pip install tqdm==4.66.2

In [ ]:
import logging
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, LoggingHandler
from torch.utils.data import DataLoader
from tqdm import tqdm


logging.basicConfig(
    format='%(asctime)s-%(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO,
    handlers=[LoggingHandler()]
)

# Implementation

**NOTE:** We need to shield our code with if `__name__`. Otherwise, CUDA runs into issues when spawning new processes. 

In [ ]:
if __name__ =='__main__':
    # size of the data that is loaded into memroy at once
    data_stream_size=16384
    # size of the chunks that are sent to each process
    chunk_size=1024
    # batch size of the model
    encode_batch_size=128
    
    # Load a large dataset in streaming mode
    ds=load_dataset('yahoo_answers_topics', split='train', streaming=True)
    dataloader=DataLoader(ds.with_format('torch'), batch_size=data_stream_size)
    
    # define the model
    model=SentenceTransformer('all-MiniLM-L6-v2')
    
    
    # start the multi-process pool on all available CUDA devices
    pool=model.start_multi_process_pool()
    
    for i, batch in enumerate(tqdm(dataloader)):
        # compute the embeddings using the multi-process pool
        sentences=batch['best_answer']
        batch_emb=model.encode_multi_process(sentences, pool, chunk_size=chunk_size, batch_size=encode_batch_size)
        print('Embeddings computed for 1 batch. Shape:', batch_emb.shape)
    # optional: stop the processes in the pool
    model.stop_multi_process_pool(pool)